## Семинар 1 Индекс

## Intro

Дедлайн - 18.09.2019

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

## Реализация:

In [192]:
import os
import numpy as np
import pandas as pd
import re

Импортируем всё нужное для работы с русским языком:

In [4]:
from pymorphy2 import analyzer

In [5]:
preprocessor = analyzer.MorphAnalyzer()

In [6]:
from pymorphy2.tokenizers import simple_word_tokenize as tokenize

In [7]:
parse = preprocessor.parse

Собираем тексты из файлов:

In [93]:
### _check : в коллекции должно быть около 165 файлов
def collect_texts(folder):
    texts, episode_names = [], []
    for root, directories, files in os.walk(folder):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as inp:
                    texts.append(inp.read())
                    episode_names.append(file[:-4])
    episode_names = np.array(episode_names)
    return texts, episode_names

In [94]:
texts, episode_names = collect_texts('friends')

Обучаем CountVectorizer:

In [8]:
## Функция препроцессинга данных:
def preprocess(string):
    tokens = [w.normal_form for w in [parse(word)[0] for word in tokenize(string)] if w.tag.POS]
    return tokens

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
vectorizer = CountVectorizer(tokenizer=preprocess)

In [13]:
vectorizer.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function preprocess at 0x000001837E007BF8>,
        vocabulary=None)

In [14]:
len(vectorizer.vocabulary_)

15129

In [15]:
len(texts)

165

Строим матрицу-индекс:

In [18]:
from sklearn.preprocessing import normalize

In [100]:
## Функция индексирования данных:
def get_index(preprocessed_texts):
    return vectorizer.transform(preprocessed_texts)

In [101]:
index = get_index(texts)

Реализуем собственно поиск:

In [105]:
## функция метрики релевантности - функция, строящая матрицу косинусной близости:
def sim_matrix(strings):
    query = normalize(vectorizer.transform(strings))
    return query * normalize(index).transpose(copy=True)

In [106]:
## функция поиска:
def multiple_search(queries):
    results = []
    if type(queries) == str:
        queries = [queries]
    sims = sim_matrix(queries).todense()
    for row in sims:
        query = np.asarray(row)[0]
        query = np.argsort(query)[::-1].tolist()
        results.append(episode_names[query])
    return results

In [170]:
multiple_search(['начать отношения', 'купить дом'])

[array(["Friends - 5x17 - The One With Rachel's Inadvertent Kiss.ru",
        'Friends - 2x16 - The One Where Joey Moves Out.ru',
        "Friends - 4x05 - The One With Joey's New Girlfriend.ru",
        "Friends - 6x21 - The One Where Ross Meets Elizabeth's Dad.ru",
        "Friends - 7x24-25 - The One With Chandler And Monica's Wedding (2).ru",
        'Friends - 6x04 - The One Where Joey Loses His Insurance.ru',
        'Friends - 3x16 - The One With The Morning After (2).ru',
        "Friends - 6x19 - The One With Joey's Fridge.ru",
        'Friends - 6x24 - The One With The Proposal (1).ru',
        "Friends - 7x07 - The One With Ross's Library Book.ru",
        'Friends - 3x05 - The One With Frank Jr..ru',
        'Friends - 4x01 - The One With The Jellyfish.ru',
        'Friends - 4x01 - The One With The Jellyfish.Tv.ru',
        "Friends - 7x18 - The One With Joey's Award.ru",
        'Friends - 4x10 - The One With The Girl From Poughkeepsie.ru',
        'Friends - 3x18 - The O

## С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [115]:
def get_inverse_index(preprocessed_texts):
    return get_index(preprocessed_texts).transpose()

In [116]:
inverse_index = get_inverse_index(texts)

In [ ]:
vectorizer.vocabulary_

In [136]:
def most_frequent(inv_index, vocab):
    freq_dict = freq_counts(inv_index, vocab)
    return max(freq_dict, key=lambda x: freq_dict[x])

In [137]:
def least_frequent(inv_index, vocab):
    freq_dict = freq_counts(inv_index, vocab)
    return sorted(freq_dict, key=lambda x: freq_dict[x])[0]

In [147]:
def freq_counts(inv_index, vocab):
    word_list = sorted([word for word in vocab], key= lambda x: vocab[x])
    freqs = np.array(inverse_index.sum(axis=1)).flatten()
    freqdict = {word: freq for word, freq in zip(word_list, freqs)}
    return freqdict

### a.) Найдём самое частотное слово:

In [148]:
most_frequent(inverse_index, vectorizer.vocabulary_)

'я'

### b.) Найдём самое редкое слово:

In [150]:
least_frequent(inverse_index, vectorizer.vocabulary_)

'-вский'

### c.) Найдём слова, которые есть во всех документах коллекции:

In [167]:
def all_doc_words(inv_index, vocab):
    words = []
    word_list = sorted([word for word in vocab], key= lambda x: vocab[x])
    for row, word in zip(inv_index, word_list):
        if len(row.nonzero()[1]) == row.shape[1]:
            words.append(word)
    return words

In [169]:
print(all_doc_words(inverse_index, vectorizer.vocabulary_))

['а', 'быть', 'в', 'весь', 'да', 'думать', 'если', 'ещё', 'знать', 'и', 'как', 'мой', 'мочь', 'мы', 'на', 'не', 'нет', 'но', 'ну', 'о', 'он', 'она', 'просто', 'с', 'сказать', 'так', 'такой', 'тот', 'ты', 'у', 'хотеть', 'что', 'это', 'этот', 'я']


### d) Какой сезон был самым популярным у Чендлера? у Моники?

In [201]:
def most_popular_season(word, inv_index, vocab, episode_names):
    season_codes = set([re.search('[0-9]+x',i).group(0) for i in episode_names])
    word_list = sorted([word for word in vectorizer.vocabulary_],
                       key= lambda x: vectorizer.vocabulary_[x])
    episode_counts = pd.DataFrame(np.array(inverse_index.todense()),
                                  index=word_list,
                                  columns=episode_names).loc[word]
    season_counts = {i:0 for i in season_codes}
    for scode in season_codes:
        season_counts[scode] = sum([episode_counts[ep] for ep in episode_names if scode in ep])
    return int(max(season_counts, key=lambda x: season_counts[x])[:-1])

In [202]:
most_popular_season('чендлер', inverse_index, vectorizer.vocabulary_, episode_names)

6

In [203]:
most_popular_season('моника', inverse_index, vectorizer.vocabulary_, episode_names)

7

### e) Кто из главных героев статистически самый популярный?

In [215]:
def most_popular_character(char_list, inv_index, vocab):
    char_counts = {char:inv_index[vocab[char]].sum() for char in char_list}
    return max(char_counts, key=lambda x: char_counts[x])

In [216]:
most_popular_character(['рэйчел','моника','фиби','джо','чендлер','росс'],
                      inverse_index,
                      vectorizer.vocabulary_)

'росс'